In [2]:
import sys
sys.path.append("../")

In [3]:
%load_ext autoreload
%autoreload 2

In [7]:
!pip install git+https://github.com/IINemo/isanlp_srl_framebank.git


  Cloning https://github.com/IINemo/isanlp_srl_framebank.git to c:\users\rukis\appdata\local\temp\pip-req-build-vgydewx1
  Resolved https://github.com/IINemo/isanlp_srl_framebank.git to commit 0c7978083911e0e539e9cf7f79e3f863c3876ee8


  Running command git clone --filter=blob:none --quiet https://github.com/IINemo/isanlp_srl_framebank.git 'C:\Users\rukis\AppData\Local\Temp\pip-req-build-vgydewx1'
  Running command git submodule update --init --recursive -q
  fatal: unable to connect to github.com:
  github.com[0: 140.82.121.4]: errno=Unknown error

  fatal: clone of 'git://github.com/IINemo/brat.git' into submodule path 'C:/Users/rukis/AppData/Local/Temp/pip-req-build-vgydewx1/docker/demo/brat' failed
  Failed to clone 'docker/demo/brat'. Retry scheduled
  fatal: unable to connect to github.com:
  github.com[0: 140.82.121.4]: errno=Unknown error

  fatal: clone of 'git://github.com/IINemo/brat.git' into submodule path 'C:/Users/rukis/AppData/Local/Temp/pip-req-build-vgydewx1/docker/demo/brat' failed
  Failed to clone 'docker/demo/brat' a second time, aborting
  error: subprocess-exited-with-error
  
  git submodule update --init --recursive -q did not run successfully.
  exit code: 1
  
  See above for output.
  
  

In [10]:
import requests
import json
import time

from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [9]:
!pip install protobuf==3.20.*

  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\rukis\\anaconda3\\Lib\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [12]:
!curl -d "{\"yandexPassportOauthToken\":\"y0_AgAAAAAFAhuyAATuwQAAAADTA2uLt9P4wIhIRcycxMNG36QNKa9u_pc\"}" "https://iam.api.cloud.yandex.net/iam/v1/tokens" --ssl-no-revoke -o iamToken.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   343  100   254  100    89   1316    461 --:--:-- --:--:-- --:--:--  1786


In [13]:
import json
with open('iamToken.txt') as f:
    d = json.loads(f.read())
    IAM_TOKEN = d['iamToken']

In [14]:
with open('folder_id.txt') as f:
    folder_id = f.read()
IAM_TOKEN

't1.9euelZqZjJPOjMnGmMfHj4yOzZLLzO3rnpWam46Pjp6Qj5uam5KQxo_Pxo3l8_ccNXpi-e8MBh4H_d3z91xjd2L57wwGHgf9.q0JdZc5jIWPffN__nS4EP94c_WCt5sLvpRRUrea527rec4Lt_Yf_m8g-PYu1g9dm_S01ZUujx8Qe8_2b0XXOCg'

In [15]:
# docker run --rm -p 3333:3333 inemo/isanlp
# docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

In [16]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault  
#
ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
res = ppl('Мы поехали на дачу.')


In [17]:
%%time
languages = ['tat', 'kaz', 'kir', 'bak', 'uzb'] # языки татарский, казахский, киргизский
language = languages[0]
url = 'https://beta.apertium.org/index.eng.html#analysis?aLang=' + language + '&aQ='    
browser = webdriver.Chrome()
browser.get(url)


CPU times: total: 0 ns
Wall time: 2.55 s


In [18]:
class Model_srl2:
    def __init__(self, RUS_SENTENCE=None):
        self.RUS_SENTENCE = RUS_SENTENCE
        self.ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
        self.language_code_dict =  {
        'татарский': ['tt', 'Tatar'],
        'казахский' : ['kk', 'Kazakh'],
        'башкирский': ['ba', 'Bashqort'],
        'киргизский' : ['ky', 'Kyrgyz']
        }

    def translation(self, texts, source_language='tt', target_language='ru'):
        #print('texts', texts)
        body = {
            "targetLanguageCode": target_language,
            "texts": texts,
            "folderId": folder_id,
            "sourceLanguageCode":  source_language
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {0}".format(IAM_TOKEN)
        }
        url = 'https://translate.api.cloud.yandex.net/translate/v2/translate'
        response = requests.post(url,
            json = body,
            headers = headers
        )
        if response.status_code != 200:
            print('Ожидаю 0.5 секунды...')  
            time.sleep(0.5)
            response = requests.post(url,
            json = body,
            headers = headers
            )
        d = json.loads(response.text)
        #print(texts, d)
        translations = d['translations']
#         print([t['text'] for t in translations])
        return [t['text'] for t in translations]
    
    def drop_punkt(self, word):
        punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
        for p in punc:
            word = word.replace(p,'')
        return word
    
    def tokenize(self, text):
        words = text.split()
        return [self.drop_punkt(word) for word in words]
    
    def get_morph(self, text, language):
        lang_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[1]/div/select')
        lang_field.send_keys(language)

        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea').clear()
        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea')

        text = self.drop_punkt(text)
        text_field.send_keys(text)

        button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[3]/div/button')
        button.click()
        sleep(1)

        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        translation = soup.find_all('td',class_="text-left")
        words = []
        for word in translation:
            word = word.text.split()
            for i, w in enumerate(word):
                if w == '↤':
                    break
            word = word[:i]
            for w in word:
                words.append(w)
        return words


    def print_roles(self, lemma, role_annot, targetLanguage=None):
        roles = []
        for sent_num, ann_sent in enumerate(role_annot):
            word_role = {}
            
            for event in ann_sent:
#                 print(event.pred)
                lemma_pred = lemma[sent_num][event.pred[0]]
#                 print(lemma_pred)
                if targetLanguage is not None:
                    trg_lang = self.language_code_dict[targetLanguage][0]
                    src_lang = 'ru'
                    #print('=====Pred: {}'.format(self.translation(lemma_pred, src_lang, trg_lang)[0]))
                    word_role['V'] = self.translation(lemma_pred, src_lang, trg_lang)
                else:
                    print('=====Pred: {}'.format(lemma_pred))
                for arg in event.args:
                    lemma_arg  = lemma[sent_num][arg.begin]
                    if targetLanguage is not None:
                        #print('Arg({}): {}'.format(arg.tag, self.translation(lemma_arg, src_lang, trg_lang)[0]))
                        word_role['Arg({})'.format(arg.tag)] = self.translation(lemma_arg, src_lang, trg_lang)
                    else:
                        print('Arg({}): {}'.format(arg.tag, lemma_arg))
            roles.append(word_role)
        return roles
    
    def get_srl_turkic(self, rus_text, selected_language):
        res = ppl(rus_text)
        return self.print_roles(res['lemma'], res['srl'], selected_language)
        
    def get_tags(self, words, srl_verb):
        new_tags = []
        for word in words:
            for srl in srl_verb.keys(): 
                if self.drop_punkt(word) in srl_verb[srl]:
                    role = srl
                    break
                else:
                    role = 'O'
            new_tags.append(role)
        return new_tags

    def get_new_roles(self, tat_sentence, morph_sentence, roles, words):
        result = {'verbs': []}
        metric = []
        for verb in roles:
            new_roles = {}
            srl_verb = {'verb':''}
#             print(verb)
            for srl in verb.keys():
    #             morph = get_morph(verb[srl], language)[0]
                morph = verb[srl][0]
#                 print(morph)
        #         Ищем слово в предложении по морфеме
                for i in range(len(morph), 0, -1):
#                     print(i, morph[:i] in morph_sentence,morph[:i], morph_sentence)
                    if morph[:i] in morph_sentence:
                        idx = morph_sentence.index(morph[:i])
                        new_roles[srl] = words[idx]
                        if srl == 'V':
                            srl_verb['verb'] = words[idx]
                        break


        #     Считаем сколько слов правильно перевели

            srl_verb['description'] = new_roles

            srl_verb['tags'] = self.get_tags(words, new_roles)

            metric.append(len(new_roles.keys()) / len(verb.keys()))
            result['verbs'].append(srl_verb)

        result['words'] = words
        result['metric'] = round(np.mean(metric),2)
        return result


    def predict(self, tat_sentence, language_ru_name, tries=5):
        pred = None
        language = self.language_code_dict[language_ru_name]

        #перевод
        rus_sentence = self.translation(tat_sentence, source_language=language[0])[0]
#         print(rus_sentence)
        # стемминг слов исходного предложения
        stem_input_words = self.get_morph(tat_sentence, language[1])
        # получить список словарей лемма - роль
        roles = self.get_srl_turkic(rus_sentence, language_ru_name)
        # Список слов в предложении
        words = tat_sentence.split()

        for i in range(tries):
            try:
                pred = self.get_new_roles(tat_sentence, stem_input_words, roles, words)
                break
            except:
                continue
        return pred
    


In [19]:
srl = Model_srl2()
with open('text_tt.txt', encoding='utf-8') as f:
    texts_tt = f.readlines()
selected_language = 'татарский'
result = srl.predict(texts_tt[2], selected_language)
result

{'verbs': [{'verb': 'юды.',
   'description': {'V': 'юды.', 'Arg(агенс)': 'Әни'},
   'tags': ['Arg(агенс)', 'O', 'V', 'O', 'O', 'O']},
  {'verb': 'йөртте.',
   'description': {'V': 'йөртте.', 'Arg(агенс)': 'Әти'},
   'tags': ['O', 'O', 'O', 'Arg(агенс)', 'O', 'V']}],
 'words': ['Әни', 'рамны', 'юды.', 'Әти', 'машинаны', 'йөртте.'],
 'metric': 0.83}

In [20]:
roles = [{'V': 'юу', 'Arg(агенс)': 'әни', 'Arg(пациенс)': 'кыса'},
        {'V': 'йөртү', 'Arg(агенс)': 'әти'}]

# Validation 

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('dataset_news-commentary-v14-wmt19.en-kk_full.csv', sep='\t')

In [23]:
srl = Model_srl2()
selected_language = 'казахский'
res = []
#  Проверка что все работает
result = srl.predict(df.loc[1,'kk'], selected_language)
result

In [24]:
pad = 1345 #+ 2587

In [25]:
df = df[pad:]

In [26]:
for i in tqdm(range(pad, df.shape[0])):
    srl_labeled = srl.predict(df.loc[i,'kk'], selected_language)
    res.append(srl_labeled)
    if i % 500 == 0:
        df_res = pd.DataFrame(res)
        df_res.to_csv('kk_res.csv', index=False)


C:\Users\rukis\AppData\Local\Temp\ipykernel_7352\2945300123.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(pad, df.shape[0])):


  0%|          | 0/4801 [00:00<?, ?it/s]

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...


KeyError: 'translations'

In [30]:
def get_datasets(df_labeled, df_sentence, language):
    df_labeled = df_labeled.fillna('')
    df_sent_copy = df_sentence.copy().reset_index().loc[:,language]
    metrics = []
    for i in range(df_labeled.shape[0]):
        try:
            metrics.append(ast.literal_eval(df_labeled.loc[i,'0'])['metric'])

        except:
            metrics.append(0)
    df_labeled.columns = ['srl'+ language]
    df_labeled['metric'] = pd.Series(metrics)
    df_labeled['sentence'] = df_sent_copy.iloc[:df_labeled.shape[0]]
    return df_labeled

In [31]:
df_labeled_kk3 = pd.DataFrame(res)
m_kk3 = get_datasets(df_labeled_kk3, df.loc[:,'kk'], 'kk')

In [37]:
m_kk3

,srlkk,metric,sentence
0,,0,Бұл – тек дамушы елдердің ғана басында бар про...
1,"{'verbs': [{'verb': 'жазып', 'description': {'...",0,Қазанның 16-сында жемқорлық туралы әшкерелеуші...
2,"{'verbs': [{'verb': '', 'description': {'Arg(к...",0,Оған қастандық жасаған үш адамға ресми айып та...
3,"{'verbs': [{'verb': 'байланысты', 'description...",0,Мальта сияқты Еуропа одағына мүше ел – Словаки...
4,"{'verbs': [{'verb': '', 'description': {'Arg(п...",0,Полиция бұл іс бойынша бірнеше адамды тұтқында...
...,...,...,...
4546,"{'verbs': [{'verb': 'ісі', 'description': {'V'...",0,"Кейбір түсіндірмелер бойынша, Трамптың бос сөз..."
4547,"{'verbs': [{'verb': '', 'description': {'Arg(с...",0,Ең жоғары ықтимал деңгейге қорқыныш сезімін кө...
4548,"{'verbs': [{'verb': 'мүмкін.', 'description': ...",0,Герман Геринг Екінші дүниежүзілік соғыстан кей...
4549,,0,Бұл оңай.


In [41]:
metrics = []
for i in range(len(res)):
    if df_labeled_kk3.loc[i,0] is not None:
        metrics.append(df_labeled_kk3.loc[i,0]['metric'])

In [42]:
np.mean(metrics)

0.5601697976878613

In [43]:
len(res)

4551

In [44]:
m_kk3.to_csv("kk_res_3_large.csv", index=False)


In [45]:
m_kk3.to_excel("kk_res_3_large.xlsx", index=False)


In [47]:
df_labeled_kk3.to_csv("kk_res_3_1.csv", index=False)


In [48]:
df_labeled_kk3.to_excel("kk_res_3_1.xlsx", index=False)
